In [1]:
I_GPU = 1

In [2]:
# %load_ext autoreload
# %autoreload 2
import os
import sys
import numpy as np
import torch
import glob
from matplotlib.colors import ListedColormap
from omegaconf import OmegaConf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)
from torch_points3d.datasets.segmentation.multimodal.s3dis import S3DISFusedDataset
# from torch_points3d.datasets.segmentation.s3dis import S3DISFusedDataset
from torch_points3d.datasets.segmentation import IGNORE_LABEL

_ = torch.cuda.is_available()
_ = torch.cuda.memory_allocated()
torch.cuda.set_device(I_GPU)

In [3]:
from omegaconf import OmegaConf
from torch_points3d.utils.config import hydra_read

# Set root to the DATA drive, where the data was downloaded
# DATA_ROOT = '/mnt/fa444ffd-fdb4-4701-88e7-f00297a8e29b/projects/datasets/s3dis'  # ???
# DATA_ROOT = '/media/drobert-admin/DATA/datasets/s3dis'  # IGN DATA
# DATA_ROOT = '/media/drobert-admin/DATA2/datasets/s3dis'  # IGN DATA2
# DATA_ROOT = '/var/data/drobert/datasets/s3dis'  # AI4GEO
# DATA_ROOT = '/home/qt/robertda/scratch/datasets/s3dis'  # CNES
DATA_ROOT = '/raid/dataset/pointcloud/data/s3dis'  # ENGIE


In [ ]:
out_dir = '/workspace/projects/torch-points3d/outputs/'

# ('5cm', ___, '____',   out_dir + '____', '___'),

checkpoints = [
#     ('2cm', 1, 'Res16UNet34-L4-early',                 out_dir + '2021-10-24/22-50-54', 'xyz+rgb'),
#     ('5cm', 2, 'Res16UNet34-L4-early',                 out_dir + '2021-10-09/21-34-55', 'xyz+rgb'),
#     ('5cm', 3, 'Res16UNet34-L4-early',                 out_dir + '2021-10-09/21-35-06', 'xyz+rgb'),
#     ('5cm', 5, 'ResUNet34-L4-early',                   out_dir + '2021-10-31/10-37-18', 'xyz+rgb'),
#     ('5cm', 5, 'ResUNet34-L4-early-nogating',          out_dir + '2021-10-31/10-37-19', 'nogating'),
#     ('5cm', 2, 'Res16UNet34-mean-RGB-early',           out_dir + '2021-11-04/18-05-10', 'mean_raw_rgb'),
#     ('5cm', 2, 'Res16UNet34-L4-early',                 out_dir + '2021-11-04/18-20-54', '512x256'),
#     ('5cm', 2, 'Res16UNet34-L4-max-early',             out_dir + '2021-11-04/18-05-57', 'max'),
#     ('5cm', 2, 'Res16UNet34-L4-late-Group',            out_dir + '2021-11-05/23-03-39', 'late'),
#     ('5cm', 5, 'ResUNet34-L4-early',                   out_dir + '2021-10-31/10-37-18', 'light'),
#     ('5cm', 5, 'Res16UNet34-L4-mean-early',            out_dir + '2021-10-15/17-44-11', 'mean'),
#     ('5cm', 5, 'Res16UNet34-L4-early-group1',          out_dir + '2021-11-05/23-17-52', 'group1'),
#     ('5cm', 5, 'Res16UNet34-L4-early',                 out_dir + '2021-11-06/23-00-46', '512x256'),
#     ('5cm', 2, 'Res16UNet34-L4-mean-early',            out_dir + '2021-11-07/09-56-20', 'mean'),
#     ('5cm', 2, 'Res16UNet34-L4-early-fromscratch',     out_dir + '2021-11-08/19-06-48', 'nopretraining'),
#     ('5cm', 5, 'Res16UNet34-L4-early-fromscratch',     out_dir + '2021-11-08/19-04-07', 'nopretraining'),
#     ('5cm', 2, 'Res16UNet34-L4-early',                 out_dir + '2021-11-09/15-30-17', 'again'),
#     ('2cm', 2, 'Res16UNet34-L4-max-early',             out_dir + '2021-11-10/13-55-27', 'max'),
#     ('2cm', 5, 'Res16UNet34-L4-max-early',             out_dir + '2021-11-10/14-09-39', 'max'),
#     ('2cm', 6, 'Res16UNet34-L4-max-early',             out_dir + '2021-11-11/21-18-44', 'max'),
    ('5cm', 2, 'Res16UNet34-align',                    out_dir + '2021-11-21/19-10-57', 'align'),
    ('5cm', 5, 'Res16UNet34-align',                    out_dir + '2021-11-21/18-49-15', 'align'),
]

for resolution, fold, model_name, directory, exp_name in checkpoints:
    
    exp_name = model_name if exp_name is None else exp_name
    
    print(f'Run exp={exp_name} fold={fold} resolution={resolution}')
    
    overrides = [
        f'model_name={model_name}',
        f"checkpoint_dir={directory}",
        'voting_runs=1',
        'tracker_options.full_res=True',
        'precompute_multi_scale=False',
        'num_workers=4',
        'batch_size=4',
        f'cuda={I_GPU}',
        'weight_name=last',
    ]
    
    cfg = hydra_read(overrides, config_name='eval')
    
    OmegaConf.set_struct(cfg, False)
    
    from torch_points3d.trainer import Trainer
    
    trainer = Trainer(cfg)
    
    # Update the val and test transforms to match train transforms for inference-time augmentation
    trainer._dataset.test_dataset[0].transform = trainer._dataset.train_dataset.transform
    trainer._dataset.test_dataset[0].transform_image = trainer._dataset.train_dataset.transform_image
    
    trainer._dataset.test_dataset[0].transform_image.transforms[4].use_coverage = False
    trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = int(2097152 * 2)
    trainer._dataset.test_dataset[0].transform_image.transforms[5].sigma = 0.02
    trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.saturation = [0.8, 1.2]
    trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.brightness = [0.8, 1.2]
    trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.contrast = [0.8, 1.2]
    
    trainer._cfg.voting_runs = 1
    
#     for i_try in range(4):
    for i_try in range(1):
        print(f'  run {i_try}...')
        trainer.eval(stage_name="test")
        torch.save(trainer._tracker._full_confusion, f'/raid/dataset/pointcloud/data/s3dis/results/{exp_name}_fold{fold}_{resolution}_{i_try}.pt')

Run exp=align fold=2 resolution=5cm


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

backbone.down_modules.5.image.conv
backbone.down_modules.4.image.conv
backbone.down_modules.3.image.conv
backbone.down_modules.2.image.conv
backbone.down_modules.1.image.conv
backbone
head
  run 0...


  0%|          | 0/209 [00:01<?, ?it/s]

Run exp=align fold=5 resolution=5cm


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]